# UCL-COMP0087 Project Demo

#### Setup

In [ ]:
# Setup
from google import colab
colab.drive.mount('/content/drive')

# all imports, login, connect drive
import os
from pathlib import Path
import requests
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive = build('drive', 'v3').files()

# recursively get names
def get_path(file_id):
    f = drive.get(fileId=file_id, fields='name, parents').execute()
    name = f.get('name')
    if f.get('parents'):
        parent_id = f.get('parents')[0]  # assume 1 parent
        return get_path(parent_id) / name
    else:
        return Path(name)

# change directory
def chdir_notebook():
    d = requests.get('http://172.28.0.2:9000/api/sessions').json()[0]
    file_id = d['path'].split('=')[1]
    path = get_path(file_id)
    nb_dir = 'drive' / path.parent
    os.chdir(nb_dir)
    return nb_dir

!cd /
chdir_notebook()

In [ ]:
!pip install -r requirements.txt

----
## A. DATA preprocessing
### 1) JSON->bin
Preprocess .json format Conala data and save processed .bin file to `data/conala` folder.

**Data Format**
* Source: Short StackOverflow natural language questions.
* Target: Code snippet. e.x. `pandas.read('file.csv', nrows=100)`.

**Preprocess**
* Canonicalization: Identify question specific string / variable names using RegEx and replace them with universal tokens `str_0`, `str_1` and `var_0`, `var_1`, etc.
* Lowercase, tokenization

#TODO: Understand preprocessing details and note down here.

**Data Split**
* Scraped data: This set is not human-curated. Used for pretraining.
* Train set: Human-curated training set for fine-tuning. In total 2185 gold data.
* Dev set: Held out 200 dev examples from Conala training set.
* Test set: 500 Conala testing instances.

In [1]:
mined_data_file = "data/conala-corpus/conala-mined.jsonl"
topk = 10 # number of pretraining data to be preprocessed
!python datasets/conala/dataset.py --pretrain=$mined_data_file --topk=$topk

process gold training data...
Skipped due to exceptions: 4
use mined data:  10
from file:  data/conala-corpus/conala-mined.jsonl
Skipped due to exceptions: 0
2185 training instances
200 dev instances
process testing data...
Skipped due to exceptions: 0
500 testing instances
number of word types: 1611, number of word types w/ frequency > 1: 846
number of singletons:  765
number of words not included: 986
total token count:  22275
unk token count:  1207
number of word types: 1709, number of word types w/ frequency > 1: 679
number of singletons:  1030
number of words not included: 1249
total token count:  14410
unk token count:  1468
number of word types: 1670, number of word types w/ frequency > 1: 685
number of singletons:  985
number of words not included: 1188
total token count:  34939
unk token count:  1391
generated vocabulary Vocab(source Vocabulary[size=629]words, primitive Vocabulary[size=464]words, code Vocabulary[size=486]words)
Max action len: 96
Avg action len: 23
Actions lar

In [6]:
# example of processed data.
from components.dataset import Dataset
n_example = 3
train_set = Dataset.from_bin_file("data/conala/train.gold.full.bin")
for src, tgt in zip(train_set.all_source[:n_example],train_set.all_targets[:n_example]):
    print(f'Source:{src} \nTarget:{tgt} \n')

Source:['concatenate', 'elements', 'of', 'a', 'list', 'str_0', 'of', 'multiple', 'integers', 'to', 'a', 'single', 'integer'] 
Target:sum(d * 10 ** i for i, d in enumerate(str_0[::-1])) 

Source:['convert', 'a', 'list', 'of', 'integers', 'into', 'a', 'single', 'integer'] 
Target:r = int(''.join(map(str, x))) 

Source:['convert', 'a', 'datetime', 'string', 'back', 'to', 'a', 'datetime', 'object', 'of', 'format', 'str_0'] 
Target:datetime.strptime('2010-11-13 10:33:54.227806', 'str_0') 



----
## Model
To adhere to the syntax requirements of code snippets, we use coding language independent AST to guide our generation of code.

**Code <-> Series of Actions**
* Target <-> Python AST <--asdl--> asdl AST <--> Action series.
* Target: code snippet.
* Python AST: Language dependent Abstract Syntax Tree.
* asdl: Text file that specifies the Grammar of Python3.
* asdl AST: Language independent Abstract Syntax Tree.
* action series: Series of actions needed to generate the AST.

TODO: some model graphs here? AST examples?

----
## Train

In [ ]:
!bash scripts/